In [4]:
from elasticsearch import Elasticsearch
from tqdm import tqdm

# Initialize the Elasticsearch client
es = Elasticsearch("http://localhost:9200")

index_name = "bron_2024_10_11"
unique_pairs = set()

# Get the total count of documents
total_docs = es.count(index=index_name)['count']

# Use a scroll or search_after to retrieve all documents
resp = es.search(index=index_name, body={"query": {"match_all": {}}, "size": 1000}, scroll='1m')

sid = resp['_scroll_id']
hits = resp['hits']['hits']

# Initialize tqdm with total_docs
with tqdm(total=total_docs, desc="Processing documents") as pbar:
    while hits:
        for doc in hits:
            source = doc['_source']
            location = source.get('location')
            location_name = source.get('location_name')
            if location and location_name:
                unique_pairs.add((location, location_name))
            pbar.update(1)  # Update the progress bar for each processed document

        resp = es.scroll(scroll_id=sid, scroll='1m')
        # Update the scroll ID and hits
        sid = resp['_scroll_id']
        hits = resp['hits']['hits']

# Convert to a list if needed and print
unique_pairs_list = list(unique_pairs)
for pair in unique_pairs_list:
    print(pair)

Processing documents: 100%|██████████| 3446697/3446697 [07:02<00:00, 8156.42it/s] 

('GM0141', 'Almelo')
('GM1896', 'Zwartewaterland')
('GM0180', 'Staphorst')


In [6]:
len(unique_pairs_list)

3